In [2]:
import re
import pandas as pd
from PyPDF2 import PdfReader 
import PyPDF2
from PyPDF2 import PdfFileReader

In [57]:
pdf_path = "C:\\Users\\prapa001\\OneDrive - Corporate\\Documents\\Line haul PDF\\NFI\\INVOICES\\DAYVILLE\\496-0321 NAD w.e. 04.01.23.pdf"  # Replace with your PDF's path
with open(pdf_path, 'rb') as file:
    reader = PdfReader(file)
    all_page_texts = [page.extract_text() for page in reader.pages]

# Extract text from all pages of the PDF
#all_page_texts = [pdf_reader.len(reader.pages).extract_text() for page_num in range(pdf_reader.len(reader.pages))]
all_page_texts = [page.extract_text() for page in reader.pages]

# Initialize text variable outside the 'with' block
text = ""
# Open the PDF file
with open(pdf_path, "rb") as file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)
    
    # Extract text from all pages using the updated methods to get pages
    for page in pdf_reader.pages:
        text += page.extract_text()

In [58]:
# Split the text using the "Pro:" pattern
invoices = re.split(r'Pro:', text)

# Since the split will use "Pro:" as a delimiter, it will be removed from the resulting text. 
# We'll add it back to the beginning of each invoice (except the first chunk, which is likely not an invoice).
invoices = [invoices[0]] + ["Pro:" + invoice for invoice in invoices[1:]]

# Display the beginning of each invoice to confirm
beginnings = [invoice[:500] for invoice in invoices]

# Remove '\n\n' from the invoices
cleaned_invoices = [invoice.replace('\n\n', ' ') for invoice in invoices]

# Function to extract specific details for each 'Pro' number
def extract_invoice_details(text):
    pro_number = re.search(r'\bPro:\s*(\d+)', text)
    bol = re.search(r'\bBOL#:\s*(\d+)', text)
    total_charges = re.search(r'Total \n\nCharges: \n\n(\$[\d,.]+)', text)
    
    pro_number = pro_number.group(1) if pro_number else None
    bol = bol.group(1) if bol else None
    total_charges = total_charges.group(1) if total_charges else None

    return pro_number, bol, total_charges

# Function to extract route details
def extract_route(text):
    # Extracting route details using regex patterns
    route_pattern = r'(\d+)?\s*\n\n?([PD])?\s*\n\n?(.*?)\s*\n\n?([\d.]+)\s*\n\n?(.*?)\s*\n\n?([A-Z]{2})\s*\n\n?(\d{5})'    
    route_matches = re.findall(route_pattern, text)
    return route_matches

# Extracting details and routes for each 'Pro' number
data = []
for detail in cleaned_invoices:
    pro_number, bol, total_charges = extract_invoice_details(detail)
    route = extract_route(detail)
    
    for r in route:
        seq_num, type_, location, distance, city, state, zip_code = r
        data.append([pro_number, bol, total_charges, seq_num, type_, location, distance, city, state, zip_code])

# Creating the dataframe
df_extracted = pd.DataFrame(data, columns=['Pro Number', 'BOL#', 'Total Charges', 'Seq Num', 'Type', 'Location', 'Distance', 'City', 'State', 'ZIP'])

df_extracted 

,Pro Number,BOL#,Total Charges,Seq Num,Type,Location,Distance,City,State,ZIP
0,11578620,1761182,None,1,P,STAPLES FC,0.0,Plainfield,CT,06374
1,11578620,1761182,None,2,D,3909AA,94.0,Lawrence,MA,01843
2,11578620,1761182,None,3,D,STAPLES DC,17.0,Putnam,CT,06260
3,11578620,1761182,None,4,P,STAPLES DC,0.0,Putnam,CT,06260
4,11578620,1761182,None,5,D,3858D,46.0,East Hartford,CT,06108
...,...,...,...,...,...,...,...,...,...,...
412,11647988,1777291,None,4,D,DICOM COURIER,27.0,Hicksville,NY,11801
413,11647988,1777291,None,5,D,STAPLES FC,160.0,Plainfield,CT,06374
414,11675430,1784370,None,2,P,STAPLES FC,14.0,Plainfield,CT,06374
415,11675430,1784370,None,3,D,3858C,43.0,East Hartford,CT,06108


In [60]:
# Validating the extracted invoices at Pro: level 
#df_extracted.nunique()
# Filter the dataframe for the specific Pro number 11252093
df_specific_pro_example = df_extracted[df_extracted["Pro Number"] == "11578620"]
df_specific_pro_example#.sum()

,Pro Number,BOL#,Total Charges,Seq Num,Type,Location,Distance,City,State,ZIP
414,11675430,1784370,None,2,P,STAPLES FC,14.0,Plainfield,CT,06374
415,11675430,1784370,None,3,D,3858C,43.0,East Hartford,CT,06108
416,11675430,1784370,None,4,D,STAPLES FC,58.0,Plainfield,CT,06374


In [7]:
#['Pro Numbers'] = '11223187'  
with pd.ExcelWriter('C:\\Users\\prapa001\\OneDrive - Corporate\\Documents\\Line haul PDF\\Parth\\Invoice Route Excel Files\\4_08_23_Invoice_01_Test_06.xlsx') as writer:
    df_extracted.to_excel(writer, index=False)